In [1]:
import spacy
en_model = spacy.load("en_core_web_sm")

## En-Fr

In [2]:
fr_model = spacy.load('fr_core_news_sm')

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('en_fr_pos_anno.csv')
en = data['EN']
fr = data['FR']

In [5]:
from simalign import SentenceAligner
    
# making an instance of our model.
# You can specify the embedding model and all alignment settings in the constructor.
myaligner = SentenceAligner(model="bert", token_type="bpe", matching_methods="mai")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2022-11-17 13:16:53,520 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


In [6]:
n = len(data) #Length of the dataset

In [7]:
# List of alignments for all sentences
alignments = []

for i in range(n):
    # Tokenize the sentence with spacy and then turn it into a list of tokens
    src_sentence = [str(token) for token in en_model(en[i])]
    trg_sentence = [str(token) for token in fr_model(fr[i])]

    # The source and target sentences should be tokenized to words.
    als = myaligner.get_word_aligns(src_sentence, trg_sentence)
    
    # The alignment is represented in the format like 0-0 1-1 ... where 0...m are indeces of the words.
    # The first index is from the source sentence, the second one is from the target
    alignments.append(' '.join(map(lambda x: f'{x[0]}-{x[1]}', als['itermax'])))
    

In [13]:
data['WA'] = alignments

In [40]:
data.to_csv('en_fr_pos_wa_anno.csv')

In [12]:
no_pos_data = data[['EN', 'FR', 'WA']]
no_pos_data.to_csv('en_fr_wa.csv')